<a id='top'></a>
# Table of contents
- [Environment](#environment)
- [Machine](#machine)
- [Dispatcher](#dispatcher)
- [Operation](#operation)
- [Job](#job)
- [Logic Test](#logic_test)

# To-Do:
- [x] ~~implement routing logic in objects~~
- [x] ~~add operation starting and end points~~
- [ ] add setting of job/machine states in machine logic
- [ ] add sources and sinks
    - source: generates new job with given intervals
    - sink: destroys jobs and finalises data collection
- [ ] add physical buffers
- [ ] add Gantt chart visualisation for debugging
- [x] ~~register job object in dispatcher~~
- [x] ~~register operations in dispatcher~~
- [ ] self-marking as disposable by jobs
    - includes demarking
    - currently only addition implemented
- [x] ~~add uniqueness check for custom IDs in resource objects (only interface to user)~~
- [x] ~~operations list of jobs as deque~~
- [ ] add status info to job DB? (waiting, processing, ...) (disposable = waiting?)
    - advantage: 
        - one central DB with all information, no cluttered information
        - simple filtering for jobs by current status incl. disposable jobs
- [ ] tracking disjunctive graph model

In [147]:
import numpy as np
import numpy.typing as npt
import simpy
import salabim as sim
from typing import TypeAlias
from collections import OrderedDict
from collections import deque
import logging
import sys
import pandas as pd
from pandas import DataFrame
from pandas import Series
from functools import lru_cache
from plotly.graph_objs._figure import Figure

# type aliases
SimPyEnv: TypeAlias = simpy.core.Environment
SalabimEnv: TypeAlias = sim.Environment
EnvID: TypeAlias = int
JobID: TypeAlias = int
OpID: TypeAlias = int
MachineID: TypeAlias = int | str
CustomID: TypeAlias = int | str
InfstructObj: TypeAlias = object # better naming in future
PlotlyFigure: TypeAlias = Figure

# forward reference, referenced before assignment
#Job: TypeAlias = 'Job'
#Dispatcher: TypeAlias = 'Dispatcher'

# logging
# IPython compatibility
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
#LOGGING_LEVEL = 'DEBUG'
logger = logging.getLogger('base')
#logger.setLevel(LOGGING_LEVEL)

In [2]:
def gen_rnd_JSSP_inst(
    n_jobs: int,
    n_machines: int,
    seed: int = 42,
) -> tuple[int, int, int, npt.NDArray[np.uint16], npt.NDArray[np.uint16], npt.NDArray[np.uint16]]:
    """
    Generates random job shop instance with given number of jobs and machines'
    - each job on all machines
    - max processing time = 9
    
    Output:
        - n_jobs: number of jobs
        - n_machines: number of machines
        - n_tasks: number of tasks
        - mat_ProcTimes: matrix of processing times | shape=(n_jobs,n_machines)
        - mat_JobMachID: matrix of machine IDs per job starting by index 1 | shape=(n_jobs,n_machines)
        - mat_OpID: matrix of operation IDs starting by index 1 | shape=(n_jobs,n_machines)
    """
    # generate random process time matrix shape=(n_jobs, n_machines)
    np_rnd_gen = np.random.default_rng(seed=seed)
    mat_ProcTimes = np_rnd_gen.integers(1, 10, size=(n_jobs,n_machines), dtype=np.uint16)
    
    # generate randomly shuffled job machine combinations
    # machine IDs from 1 to n_machines
    temp = np.arange(0, (n_machines), step=1, dtype=np.uint16)
    temp = np.expand_dims(temp, axis=0)
    # repeat dummy line until number n_jobs is reached
    temp = np.repeat(temp, n_jobs, axis=0)
    # randomly permute the machine indices job-wise
    mat_JobMachID = np_rnd_gen.permuted(temp, axis=1)
    
    # generate operation ID matrix
    n_ops = n_jobs * n_machines
    temp2 = np.arange(0, (n_ops), step=1, dtype=np.uint16)
    mat_OpID = temp2.reshape(n_jobs, -1)
    
    return n_jobs, n_machines, n_ops, mat_ProcTimes, mat_JobMachID, mat_OpID

In [3]:
(n_jobs, n_machines, n_ops, 
 mat_ProcTimes, mat_JobMachID, mat_OpID) = gen_rnd_JSSP_inst(2,3)

In [4]:
mat_ProcTimes

array([[2, 1, 9],
       [7, 9, 6]], dtype=uint16)

In [5]:
mat_JobMachID

array([[2, 0, 1],
       [0, 1, 2]], dtype=uint16)

##### Brainstorming of ways to import machine names and IDs
*Case 1: only names are given*
- read machine names and assert IDs to them
- build data structure with name and ID bundled (maybe as property of a machine class)

*Case 2: IDs are given*
- only building data structure with name and ID bundled

*Data Structure:*
- if only mapping of two pairs in each direction (lookup ID or lookup machine name)
    - bi-directional dictionary

**for resource objects:**
- CustomID may not be of type 'None' because the custom identifiers are the only interface to the end user (EnvID are solely handled inernally)
- add checking for uniqueness of custom identifiers necessary, else the mapping of different objects could be ambiguous
- jobs and operations can use ambiguous custom IDs --> custom IDs can still be None

- dedicated environment class with information on associated resources and jobs
- maybe add possibility of using subsystems (bundle of resources with unique identifiers)

In [6]:
class SimulationEnvironment(simpy.core.Environment):
    Dispatcher: TypeAlias = 'Dispatcher'
    
    def __init__(
        self,
        *args,
        **kwargs,
    ) -> None:
        """
        
        """
        super().__init__(*args, **kwargs)
        
        # resource data base as simple Pandas DataFrame
        self._infstruct_prop: dict[str, type] = {
            'env_id': int,
            'custom_id': object,
            'resource': object,
            'name': str,
            'res_type': str,
        }
        self._res_db: DataFrame = pd.DataFrame(columns=list(self._infstruct_prop.keys()))
        self._res_db: DataFrame = self._res_db.astype(self._infstruct_prop)
        self._res_lookup_props: set[str] = set(['env_id', 'custom_id', 'name'])
        
        # env identifiers
        self._id_counter: EnvID = 0
        self._res_custom_identifiers: set[str | int] = set()
        
        # job dispatcher
        self._dispatcher_registered: bool = False
        self._dispatcher: Dispatcher = None
        
        ############## LEGACY CODE
        ### legacy code, changed approach to tabular data structure
        """
        self.id_counter: EnvID = 0
        self.resources: dict[EnvID, object] = dict()
        self._custom_identifiers: set[CustomID] = set()
        self._custom_from_env_ids: dict[EnvID, CustomID] = dict()
        self._custom_to_env_ids: dict[CustomID, EnvID] = dict()
        """
        
    def _obtain_env_id(self) -> EnvID:
        """Simple counter function for managing environment IDs"""
        # assign id and set counter up
        env_id = self._id_counter
        self._id_counter += 1
        
        return env_id
    
    def register_dispatcher(
        self,
        dispatcher: Dispatcher,
    ) -> EnvID:
        """
        Registers a dispatcher instance for the environment. Only one instance per environment is allowed.
        returns: EnvID for the dispatcher instance
        """
        # obtain env_id
        env_id = self._obtain_env_id()
        
        if not self._dispatcher_registered:
            self._dispatcher = dispatcher
            self._dispatcher_registered = True
            logger.info(f"Successfully registered dispatcher with EnvID {env_id}")
        else:
            raise AssertionError("There is already a registered dispatcher instance \
                                 Only one instance per environement is allowed.")
        
        return env_id
    
    @property
    def dispatcher(self) -> Dispatcher:
        """obtain the current registered dispatcher instance of the environment"""
        if self._dispatcher is None:
            raise ValueError("No Dipsatcher instance registered.")
        else:
            return self._dispatcher
    
    def register_resource(
        self,
        obj: InfstructObj,
        custom_identifier: CustomID,
        name: str | None,
    ) -> tuple[EnvID, str]:
        """
        registers an infrastructure object in the environment by assigning an unique id and 
        adding the object to the associated resources of the environment
        
        object:     env resource
        returns:
            env_id: assigned env ID
        """
        # check for uniqueness of custom_identifier
        # type security
        if not isinstance(custom_identifier, (str, int)):
            raise TypeError("Custom identifier must be of type STR or INT")
        # create check value
        if isinstance(custom_identifier, str):
            # remove capital letters for checking
            check_val = custom_identifier.lower()
        else:
            check_val = custom_identifier
        
        # check if value already exists
        if check_val in self._res_custom_identifiers:
            raise ValueError(f"The custom identifier {custom_identifier} provided already exists, \
                            but has to be unique.")
        else:
            self._res_custom_identifiers.add(check_val)
        
        # obtain env_id
        env_id = self._obtain_env_id()
        
        # custom name
        if name is None:
            name = f'M_env_{env_id}'
        
        # new entry for resource data base
        new_entry: DataFrame = pd.DataFrame({
                                'env_id': [env_id],
                                'custom_id': [custom_identifier],
                                'resource': [obj],
                                'name': [name],
                                'res_type': [obj.res_type]})
        new_entry: DataFrame = new_entry.astype(self._infstruct_prop)
        self._res_db = pd.concat([self._res_db, new_entry], ignore_index=True)
        
        logger.info(f"Successfully registered object with EnvID {env_id} and name {name}")
        
        return env_id, name
    
    @property
    def res_db(self) -> DataFrame:
        """obtain a current overview of registered objects in the environment"""
        return self._res_db

    #@lru_cache(maxsize=200)
    def get_res_obj_by_prop(
        self,
        property: str, 
        val: EnvID | CustomID | str,
        target_prop: str = 'resource',
    ) -> InfstructObj:
        """
        obtain a resource object from the environment by its property and corresponding value
        properties: env_id, custom_id, name
        """
        # check if property is a filter criterion
        if property not in self._res_lookup_props:
            raise IndexError(f"Property '{property}' is not allowed. Choose from {self._res_lookup_props}")
        # None type value can not be looked for
        if val is None:
            raise TypeError("The lookup value can not be of type 'None'.")
        
        # filter resource database for prop-value pair
        temp1: Series = self._res_db.loc[self._res_db[property] == val, target_prop]
        # check for empty search result, at least one result necessary
        if len(temp1) == 0:
            raise IndexError(f"There were no resources found for the property '{property}' \
                            with the value '{val}'")
        # check for multiple entries with same prop-value pair
        ########### PERHAPS CHANGE NECESSARY
        ### multiple entries but only one returned --> prone to errors
        elif len(temp1) > 1:
            # warn user
            logger.warning(f"CAUTION: There are multiple resources which share the \
                        same value '{val}' for the property '{property}'. \
                        Only the first entry is returned.")
        
        return temp1.iat[0]
    
        

<a id='environment'></a>
**Salabim Env**

In [7]:
class SimulationEnvironment(sim.Environment):
    Dispatcher: TypeAlias = 'Dispatcher'
    
    def __init__(
        self,
        *args,
        **kwargs,
    ) -> None:
        """
        
        """
        super().__init__(*args, **kwargs)
        
        # resource data base as simple Pandas DataFrame
        self._infstruct_prop: dict[str, type] = {
            'env_id': int,
            'custom_id': object,
            'resource': object,
            'name': str,
            'res_type': str,
        }
        self._res_db: DataFrame = pd.DataFrame(columns=list(self._infstruct_prop.keys()))
        self._res_db: DataFrame = self._res_db.astype(self._infstruct_prop)
        self._res_lookup_props: set[str] = set(['env_id', 'custom_id', 'name'])
        
        # env identifiers
        self._id_counter: EnvID = 0
        self._res_custom_identifiers: set[str | int] = set()
        
        # job dispatcher
        self._dispatcher_registered: bool = False
        self._dispatcher: Dispatcher = None
        
        ############## LEGACY CODE
        ### legacy code, changed approach to tabular data structure
        """
        self.id_counter: EnvID = 0
        self.resources: dict[EnvID, object] = dict()
        self._custom_identifiers: set[CustomID] = set()
        self._custom_from_env_ids: dict[EnvID, CustomID] = dict()
        self._custom_to_env_ids: dict[CustomID, EnvID] = dict()
        """
        self._ultimative_test = pd.DataFrame(columns=['test1', 'test2'])
        
    def _obtain_env_id(self) -> EnvID:
        """Simple counter function for managing environment IDs"""
        # assign id and set counter up
        env_id = self._id_counter
        self._id_counter += 1
        
        return env_id
    
    def register_dispatcher(
        self,
        dispatcher: Dispatcher,
    ) -> EnvID:
        """
        Registers a dispatcher instance for the environment. Only one instance per environment is allowed.
        returns: EnvID for the dispatcher instance
        """
        # obtain env_id
        env_id = self._obtain_env_id()
        
        if not self._dispatcher_registered:
            self._dispatcher = dispatcher
            self._dispatcher_registered = True
            logger.info(f"Successfully registered dispatcher with EnvID {env_id}")
        else:
            raise AssertionError("There is already a registered dispatcher instance \
                                 Only one instance per environement is allowed.")
        
        return env_id
    
    @property
    def dispatcher(self) -> Dispatcher:
        """obtain the current registered dispatcher instance of the environment"""
        if self._dispatcher is None:
            raise ValueError("No Dipsatcher instance registered.")
        else:
            return self._dispatcher
    
    def register_resource(
        self,
        obj: InfstructObj,
        custom_identifier: CustomID,
        name: str | None,
    ) ->  tuple[EnvID, str]:
        """
        registers an infrastructure object in the environment by assigning an unique id and 
        adding the object to the associated resources of the environment
        
        object:     env resource
        returns:
            env_id: assigned env ID
        """
        # check for uniqueness of custom_identifier
        # type security
        if not isinstance(custom_identifier, (str, int)):
            raise TypeError("Custom identifier must be of type STR or INT")
        # create check value
        if isinstance(custom_identifier, str):
            # remove capital letters for checking
            check_val = custom_identifier.lower()
        else:
            check_val = custom_identifier
        
        # check if value already exists
        if check_val in self._res_custom_identifiers:
            raise ValueError(f"The custom identifier {custom_identifier} provided already exists, \
                            but has to be unique.")
        else:
            self._res_custom_identifiers.add(check_val)
        
        # obtain env_id
        env_id = self._obtain_env_id()
        
        # custom name
        if name is None:
            name = f'M_env_{env_id}'
        
        # new entry for resource data base
        new_entry: DataFrame = pd.DataFrame({
                                'env_id': [env_id],
                                'custom_id': [custom_identifier],
                                'resource': [obj],
                                'name': [name],
                                'res_type': [obj.res_type]})
        new_entry: DataFrame = new_entry.astype(self._infstruct_prop)
        self._res_db = pd.concat([self._res_db, new_entry], ignore_index=True)
        
        logger.info(f"Successfully registered object with EnvID {env_id} and name {name}")
        
        return env_id, name
    
    @property
    def res_db(self) -> DataFrame:
        """obtain a current overview of registered objects in the environment"""
        return self._res_db

    #@lru_cache(maxsize=200)
    def get_res_obj_by_prop(
        self,
        property: str, 
        val: EnvID | CustomID | str,
        target_prop: str = 'resource',
    ) -> InfstructObj:
        """
        obtain a resource object from the environment by its property and corresponding value
        properties: env_id, custom_id, name
        """
        # check if property is a filter criterion
        if property not in self._res_lookup_props:
            raise IndexError(f"Property '{property}' is not allowed. Choose from {self._res_lookup_props}")
        # None type value can not be looked for
        if val is None:
            raise TypeError("The lookup value can not be of type 'None'.")
        
        # filter resource database for prop-value pair
        temp1: Series = self._res_db.loc[self._res_db[property] == val, target_prop]
        # check for empty search result, at least one result necessary
        if len(temp1) == 0:
            raise IndexError(f"There were no resources found for the property '{property}' \
                            with the value '{val}'")
        # check for multiple entries with same prop-value pair
        ########### PERHAPS CHANGE NECESSARY
        ### multiple entries but only one returned --> prone to errors
        elif len(temp1) > 1:
            # warn user
            logger.warning(f"CAUTION: There are multiple resources which share the \
                        same value '{val}' for the property '{property}'. \
                        Only the first entry is returned.")
        
        return temp1.iat[0]
    
    
        

**Link Collection**

- [Environment](#environment)
- [Machine](#machine)
- [Dispatcher](#dispatcher)
- [Operation](#operation)
- [Job](#job)
- [Logic Test](#logic_test)


[Jump to top](#top)

<a id='machine'></a>

In [8]:
class Machine(sim.Component):
    Job: TypeAlias = 'Job'
    Operation: TypeAlias = 'Operation'
    
    def __init__(
        self,
        env: SimulationEnvironment,
        custom_identifier: CustomID,
        name: str | None = None,
        num_slots: int = 1,
        *args,
        **kwargs,
    ) -> None:
        """
        env:        SimPy Environment in which machine is embedded
        num_slots:  capacity of the machine, if multiple processing 
                    slots available at the same time > 1, default=1
        """
        
        
        ############# custom identifiers only over env_id
        ### associate env_id with custom_id in env
        ### lookup env_id of object in environment and obtain custom_id
        
        
        ############# CHECK IF NECESSARY IN FUTURE
        """
        if custom_identifier is not None:
            ret = env.register_custom_identifier(
                    env_ID=self.env_id, custom_identifier=custom_identifier)
        """
        
        
        # assert machine information and register object in the environment
        #self._env = env
        self.res_type: str = 'Machine'
        self._env_id, name = env.register_resource(
                                obj=self, custom_identifier=custom_identifier,
                                name=name)
        self.custom_identifier = custom_identifier
        
        # intialize base class
        super().__init__(env=env, name=name, *args, **kwargs)
        
        # add (logical) buffer
        # each resource uses an associated buffer, even if there is none physically available
        buff_name = f"buffer_{self.name()}"
        self.buffer = sim.Queue(name=buff_name, env=self.env)
        
        # currently processed job
        self.current_job_ID: int | None = None
        self.current_job: Job | None = None
        
        # machine state parameters
        self.is_occupied: bool = False
        self.is_waiting: bool = False
        self.is_blocked: bool = False
        self.is_failed: bool = False
        # maybe for future, curently no working time calendars planned
        self.is_paused: bool = False
        
        # time in state parameters
        self.time_occupied: float = 0.
        self.time_waiting: float = 0.
        self.time_blocked: float = 0.
        self.time_failed: float = 0.
        
        # number of inputs/outputs
        self.num_jobs_input: int = 0
        self.num_jobs_output: int = 0
    
    @property
    def env_id(self) -> EnvID:
        return self._env_id
    """
    @property
    def env(self) -> SimulationEnvironment:
        return self._env
    """
    
    ### STATE SETTING
    
    def process(self) -> None:
        while True:
            # initialise state by passivating machines
            # machine are activated by jobs
            if len(self.buffer) == 0:
                yield self.passivate()
            print(f"Machine {self.custom_identifier} is getting job from queue")
            # theoretically request to dispatcher possible
            # request job from associated queue
            
            ### REQUEST TO DISPATCHER (MACHINE SIDE: Sequencing)
            # --> job instance
            job: Job = self.buffer.pop()
            current_op: Operation = job.current_op
            # get processing time of the current operation
            proc_time: float = current_op.proc_time
            
            print(f"[START] job ID {job.job_id} at {self.env.now()} on machine ID {self.custom_identifier} \
                with proc time {proc_time}")
            
            # RELEVANT INFORMATION BEFORE PROCESSING
            current_op.start_time = self.env.now()
            # PROCESSING
            yield self.hold(proc_time)
            # RELEVANT INFORMATION AFTER PROCESSING
            current_op.end_time = self.env.now()
            current_op.is_finished = True
            job.num_finished_ops += 1
            
            print(f"[END] job ID {job.job_id} at {self.env.now()} on machine ID {self.custom_identifier}")
            
            # return back control to job instance
            job.activate()

**Link Collection**

- [Environment](#environment)
- [Machine](#machine)
- [Dispatcher](#dispatcher)
- [Operation](#operation)
- [Job](#job)
- [Logic Test](#logic_test)


[Jump to top](#top)

In [9]:
(n_jobs, n_machines, n_ops, 
 mat_ProcTimes, mat_JobMachID, mat_OpID) = gen_rnd_JSSP_inst(2,3)

In [10]:
mat_ProcTimes

array([[2, 1, 9],
       [7, 9, 6]], dtype=uint16)

In [11]:
mat_JobMachID

array([[2, 0, 1],
       [0, 1, 2]], dtype=uint16)

In [12]:
mat_OpID

array([[0, 1, 2],
       [3, 4, 5]], dtype=uint16)

In [13]:
env = SimulationEnvironment(name='base')

for machine in np.unique(mat_JobMachID):
    MachInst = Machine(env=env, custom_identifier=machine.item())

INFO:base:Successfully registered object with EnvID 0 and name M_env_0
INFO:base:Successfully registered object with EnvID 1 and name M_env_1
INFO:base:Successfully registered object with EnvID 2 and name M_env_2


In [14]:
env.res_db

,env_id,custom_id,resource,name,res_type
0,0,0,Machine (M_env_0),M_env_0,Machine
1,1,1,Machine (M_env_1),M_env_1,Machine
2,2,2,Machine (M_env_2),M_env_2,Machine


In [15]:
env

SimulationEnvironment (base)

In [16]:
MachInst.env

SimulationEnvironment (base)

In [17]:
MachInst.name()

'M_env_2'

In [18]:
MachInst.buffer

Queue (buffer_M_env_2)

In [19]:
ret = env.get_res_obj_by_prop(property='env_id', val=0)

In [20]:
ret.name()

'M_env_0'

In [21]:
ret.buffer

Queue (buffer_M_env_0)

---

#### **Adding database approach also to the job dispatcher**
**properties**:
- job_id
- custom_id
- job instance
- name
- product_type (for later implementation of different product types)

<a id='dispatcher'></a>

In [181]:
class Dispatcher(object):
    Dispatcher: TypeAlias = 'Dispatcher'
    Job: TypeAlias = 'Job'
    Operation: TypeAlias = 'Operation'
    
    def __init__(
        self,
        env: SimulationEnvironment,
    ) -> None:
        """
        Dispatcher class for given environment (only one dispatcher for each environment)
        - different functions to monitor all jobs in the environment
        - jobs report back their states to the dispatcher
        """
                
        # job data base as simple Pandas DataFrame
        # column data types
        self._job_prop: dict[str, type] = {
            'job_id': int,
            'custom_id': object,
            'job': object,
            'name': str,
            'job_type': str,
            'status': str,
        }
        self._job_db: DataFrame = pd.DataFrame(columns=list(self._job_prop.keys()))
        self._job_db: DataFrame = self._job_db.astype(self._job_prop)
        self._job_db: DataFrame = self._job_db.set_index('job_id')
        self._job_lookup_props: set[str] = set(['job_id', 'custom_id', 'name'])
        
        # operation data base as simple Pandas DataFrame
        # column data types
        self._op_prop: dict[str, type] = {
            'op_id': int,
            'job_id': int,
            'job_name': str,
            'custom_id': object,
            'op': object,
            'name': str,
            'machine': object,
            'machine_name': str,
            'duration': float,
            'start_time': float,
            'end_time': float,
            'status': str,
        }
        self._op_db: DataFrame = pd.DataFrame(columns=list(self._op_prop.keys()))
        self._op_db: DataFrame = self._op_db.astype(self._op_prop)
        self._op_db: DataFrame = self._op_db.set_index('op_id')
        self._op_lookup_props: set[str] = set(['op_id', 'job_id', 'custom_id', 'name', 'machine'])
                
        # register in environment and get EnvID
        self._env = env
        self._env_id: EnvID = self._env.register_dispatcher(self)
        
        ########## PERHAPS REWORK
        self._disposable_jobs: dict[int, Job] = dict()
        self.job_pool: OrderedDict[JobID, Job] = OrderedDict()
        # managing IDs
        self._id_types = set(['job', 'op'])
        self._job_id_counter: JobID = 0
        self._op_id_counter: OpID = 0
    
    @property
    def env_id(self) -> EnvID:
        return self._env_id
    
    @property
    def env(self) -> SimulationEnvironment:
        return self._env
    
    def _obtain_job_id(self) -> JobID:
        """Simple counter function for managing job IDs"""
        # assign id and set counter up
        job_id = self._id_counter
        self._id_counter += 1
        
        return job_id
    
    def _obtain_op_id(self) -> OpID:
        """Simple counter function for managing operation IDs"""
        # assign id and set counter up
        op_id = self._op_id_counter
        self._op_id_counter += 1
        
        return op_id
    
    def _obtain_load_obj_id(
        self,
        load_type: str,
    ) -> JobID | OpID:
        """Simple counter function for managing operation IDs"""
        # assign id and set counter up
        
        if load_type not in self._id_types:
            raise ValueError(f"Given type {type} not valid. Choose from '{self._id_types}'")
        
        match load_type:
            case 'job':
                ident_no = self._job_id_counter
                self._job_id_counter += 1
            case 'op':
                ident_no = self._op_id_counter
                self._op_id_counter += 1
        
        return ident_no
    
    def register_job(
        self,
        obj: Job,
        custom_identifier: CustomID | None,
        name: str | None,
        status: str | None,
    ) -> tuple[SimulationEnvironment, JobID, str]:
        """
        registers an job object in the dispatcher instance by assigning an unique id and 
        adding the object to the associated jobs
        
        object:     env resource
        returns:
            env_id: assigned env ID
        """
        # obtain id
        job_id = self._obtain_load_obj_id(load_type='job')
        
        # custom name
        if name is None:
            name = f'J_gen_{job_id}'
        
        # new entry for job data base
        new_entry: DataFrame = pd.DataFrame({
                                'job_id': [job_id],
                                'custom_id': [custom_identifier],
                                'job': [obj],
                                'name': [name],
                                'job_type': [obj.job_type],
                                'status': status})
        new_entry: DataFrame = new_entry.astype(self._job_prop)
        new_entry: DataFrame = new_entry.set_index('job_id')
        self._job_db: DataFrame = pd.concat([self._job_db, new_entry])
        
        logger.info(f"Successfully registered job with JobID {job_id} and name {name}")
        
        return self.env, job_id, name
    
    def register_operation(
        self,
        obj: Operation,
        machine_identifier: MachineID,
        custom_identifier: CustomID | None,
        name: str | None,
        status: str | None, ###### add status types later
    ) -> tuple[OpID, str, Machine]: ##### add machine type later
        """
        registers an operation object in the dispatcher instance by assigning an unique id and 
        adding the object to the associated operations
        
        obj: operation to register
        machine_identifier: custom ID of the associated machine (user interface)
        custom_identifier: custom identifier of the operation 
            (kept for consistency reasons, perhaps remove later)
        name: assigned name the operation
        status: for future features if status of operations is tracked
        
        outputs:
        op_id: assigned operation ID
        name: assigned name
        machine: corresponding machine infrastructure object
        """
        # obtain id
        op_id = self._obtain_load_obj_id(load_type='op')
        
        # custom name
        if name is None:
            name = f'O_gen_{op_id}'
        
        # corresponding machine object on which operation is performed
        machine = self._env.get_res_obj_by_prop(property='custom_id', val=machine_identifier)
        
        # new entry for operation data base
        new_entry: DataFrame = pd.DataFrame({
                                'op_id': [op_id],
                                'job_id': [obj.job_id],
                                'job_name': [obj.job.name()],
                                'custom_id': [custom_identifier],
                                'op': [obj],
                                'name': [name],
                                'machine': [machine],
                                'machine_name': [machine.name()],
                                'duration': [obj.proc_time],
                                'start_time': [obj.start_time],
                                'end_time': [obj.end_time],
                                'status': status})
        new_entry: DataFrame = new_entry.astype(self._op_prop)
        new_entry: DataFrame = new_entry.set_index('op_id')
        self._op_db: DataFrame = pd.concat([self._op_db, new_entry])
        
        logger.info(f"Successfully registered operation with OpID {op_id} and name {name}")
        
        # return machine object
        return op_id, name, machine
    
    def write_op_times(
        self,
        op_id: OpID,
    ) -> None:
        """
        add start and end time information to the operation database 
        """
        # get operation instance
        curr_op: Operation = self._op_db.loc[op_id, 'op']
        # write operation properties to operation database
        self._op_db.loc[op_id, 'start_time'] = curr_op.start_time
        self._op_db.loc[op_id, 'end_time'] = curr_op.end_time
    
    # special function for custom random generator of JSSP instances
    def gen_job_pool_generic(
        self,
        mat_ProcTimes: npt.NDArray[np.uint16],
        mat_JobMachID: npt.NDArray[np.uint16],
    ) -> OrderedDict[JobID, Job]:
        """
        function to build a integrated job pool if generic JxM JSSP instances are used
        mat_ProcTimes: matrix of processing times | shape=(n_jobs,n_machines)
        mat_JobMachID: matrix of machine IDs per job starting by index 1 | shape=(n_jobs,n_machines)
        mat_OpID: matrix of operation IDs starting by index 1 | shape=(n_jobs,n_machines)
        """
            
        for job_id in range(len(mat_ProcTimes)):
            temp1 = mat_ProcTimes[job_id].tolist()
            temp2 = mat_JobMachID[job_id].tolist()
            JobInst = Job(
                dispatcher=self,
                proc_times=temp1,
                machine_order=temp2,
                custom_identifier=None,
                name=None,
            )
            ######### NOT NECESSARY ANYMORE BECAUSE JOBS REGISTER THEMSELVES
            # IN THE DISPATCHER INSTANCE AND THERFORE IN THE JOB DATABASE
            ######### ADD TO JOB (bottom-up approach) #######################
            ### jobs add themselves to job pool
            self.job_pool[job_id] = JobInst
            
        return self.job_pool
     
    @property
    def job_db(self) -> DataFrame:
        """
        obtain a current overview of registered jobs in the environment
        """
        return self._job_db
    
    @property
    def op_db(self) -> DataFrame:
        """
        obtain a current overview of registered operations in the environment
        """
        return self._op_db

    #@lru_cache(maxsize=200)
    def get_job_obj_by_prop(
        self,
        property: str, 
        val: EnvID | CustomID | str,
        target_prop: str = 'job',
    ) -> Job:
        """
        obtain a job object from the dispatcher by its property and corresponding value
        properties: job_id, custom_id, name
        """
        # check if property is a filter criterion
        if property not in self._job_lookup_props:
            raise IndexError(f"Property '{property}' is not allowed. Choose from {self._job_lookup_props}")
        # None type value can not be looked for
        if val is None:
            raise TypeError("The lookup value can not be of type 'None'.")
        
        # filter resource database for prop-value pair
        temp1: Series = self._job_db.loc[self._job_db[property] == val, target_prop]
        # check for empty search result, at least one result necessary
        if len(temp1) == 0:
            raise IndexError(f"There were no resources found for the property '{property}' \
                            with the value '{val}'")
        # check for multiple entries with same prop-value pair
        ########### PERHAPS CHANGE NECESSARY
        ### multiple entries but only one returned --> prone to errors
        elif len(temp1) > 1:
            # warn user
            logger.warning(f"CAUTION: There are multiple resources which share the \
                        same value '{val}' for the property '{property}'. \
                        Only the first entry is returned.")
        
        return temp1.iat[0]
    
    def draw_gantt_chart(
        self,
        sort_by_machine_name: bool = False,
        sort_ascending: bool = True,
    ) -> PlotlyFigure:
        """
        draw a Gantt chart based on the dispatcher's operation database
        sort_by_machine_name: whether to sort by machine name (True) or by job name (False) \
            default: False
        sort_ascending: whether to sort in ascending (True) or descending order (False) \
            default: True
        use_duration: plot each operation with its scheduled duration instead of the delta time \
            between start and end; if there were no interruptions both methods return the same results \
            default: False
        """
        # filter operation DB for relevant information
        df = self._op_db.filter(items=['job_name', 'machine_name', 'start_time', 'end_time', 'duration'])
        # calculate delta time between start and end
        df['delta'] = df['end_time'] - df['start_time']
        
        # sorting
        sort_key: str = ''
        if sort_by_machine_name:
            sort_key = 'machine_name'
        else:
            sort_key = 'job_name' 
        
        df = df.sort_values(by=sort_key, ascending=sort_ascending, kind='stable')
        
        # build Gantt chart with Plotly Timeline
        fig = px.timeline(df, x_start='start_time', x_end='end_time', 
                          y='machine_name', color='job_name')
        fig.update_yaxes(autorange='reversed')
        fig.layout.xaxis.type = 'linear'

        # reset axis scale for every figure element
        # https://stackoverflow.com/questions/66078893/plotly-express-timeline-for-gantt-chart-with-integer-xaxis
        for d in fig.data:
            filt = df['job_name'] == d.name
            d.x = df.loc[filt, 'delta']

        fig.show()
        
        return fig
    
    ### DISPOSABLE JOBS
    ### STILL NECESSARY???
    def add_disposable_job(
        self,
        job: Job,
    ) -> None:
        """
        add job to the disposable ones
        """
        self._disposable_jobs[job.job_id] = job
    
    @property
    def disposable_jobs(self) -> dict[int, Job]:
        return self._disposable_jobs
    
    ################# REWORK ##################
    ### maybe add a corresponding property in the job DB
    def get_disposable_jobs(
        self,
        job_set: OrderedDict,
    ) -> tuple[list[JobID], list[Job]]:
        """
        function needs to be reworked, jobs should report back information to a dispatcher instance
        (bottom-up instead of top-down)
        """
        #########################################
        self._disposable_jobs_ID: list[int] = list()
        self._disposable_jobs: list[Job] = list()
        
        for job_id, job in job_set.items():
            if job.is_disposable:
                self._disposable_jobs_ID.append(job_id)
                self._disposable_jobs.append(job)
                
        return self._disposable_jobs_ID, self._disposable_jobs

**Link Collection**

- [Environment](#environment)
- [Machine](#machine)
- [Dispatcher](#dispatcher)
- [Operation](#operation)
- [Job](#job)
- [Logic Test](#logic_test)


[Jump to top](#top)

### Logic behind simulation model approach
*Model of Resource and Load*:
- system consists of physical objects, also called infrastructure
    - each element can be considered as encapsulated resource
- stress can be put on each system by occupying resources, also called load
    - definition of load depends on system type and modelling, e.g. production jobs for production systems or customers for cashiers in a shop
- load objects are called ***load unit***

*Guiding Priciples*:
- **load objects** can only be spatially and temporally modified by **resources**
    - whole routing logic is implemented in a collaborative manner between resources and load objects
        - each load object puts itself in a associated queue
        - **but no load object can change its state without a associated resource**
    - load objects **contain the necessary information** which is essential for their further processing

<a id='operation'></a>

In [124]:
class Operation(object):
    Job: TypeAlias = 'Job'
    Dispatcher: TypeAlias = 'Dispatcher'
    
    def __init__(
        self,
        dispatcher: Dispatcher,
        job: Job,
        proc_time: float,
        machine_identifier: MachineID,
        custom_identifier: CustomID | None = None,
        name: str | None = None,
        status: str | None = None,
    ) -> None:
        """
        identifier: operation's ID
        proc_times: operation's processing times
        machine_identifier: ID of machine on which operation is processed
        """
        # !!!!!!!!! perhaps processing times in future multiple entries depending on associated machine
        # change of input format necessary, currently only one machine for each operation
        # no groups, no differing processing times for different machines 

        # assert operation information
        self._dispatcher = dispatcher
        self._job = job
        self._job_id = job.job_id
        
        # process information
        # processing time
        self.proc_time: float = proc_time
        # starting and end points
        # in future setting starting points in advance possible
        self.start_time: float = 0.
        self.end_time: float = 0.
        self.is_finished: bool = False
        
        ########### adding machine instances
        ### perhaps adding machine sets if multiple machines possible (machine groups)
        ### assignment of machine instance by dispatcher
        # from dispatcher: op_id, name, target_machine
        self._op_id, self.name, self.target_machine = self.dispatcher.register_operation(
                                                        obj=self, machine_identifier=machine_identifier,
                                                        custom_identifier=custom_identifier, name=name, 
                                                        status=status)  
        
    @property   
    def dispatcher(self) -> Dispatcher:
        return self._dispatcher
    
    @property
    def op_id(self) -> OpID:
        return self._op_id
    
    @property
    def job(self) -> Job:
        return self._job
    
    @property
    def job_id(self) -> JobID:
        return self._job_id

**Link Collection**

- [Environment](#environment)
- [Machine](#machine)
- [Dispatcher](#dispatcher)
- [Operation](#operation)
- [Job](#job)
- [Logic Test](#logic_test)


[Jump to top](#top)

<a id='job'></a>

In [125]:
class Job(sim.Component):
    Job: TypeAlias = 'Job'
    Dispatcher: TypeAlias = 'Dispatcher'
    
    def __init__(
        self,
        dispatcher: Dispatcher,
        proc_times: list[float],
        machine_order: list[int],
        custom_identifier: CustomID | None = None,
        name: str | None = None,
        status: str | None = None,
        *args,
        **kwargs,
    ) -> None:
        """
        ############## ADD DESCRIPTION
        """
        ### BASIC INFORMATION ###
        # assert job information
        self.custom_identifier = custom_identifier
        self.job_type: str = 'Job'
        self._dispatcher = dispatcher
        
        ### register job instance
        env, self._job_id, name = self._dispatcher.register_job(
                                    obj=self, custom_identifier=self.custom_identifier,
                                    name=name, status=status)
        
        # intialize base class
        super().__init__(env=env, name=name, *args, **kwargs)
        
        ### OPERATIONS ##
        self.operations = deque()
        
        for idx, op_proc_time in enumerate(proc_times):
            Op = Operation(
                dispatcher=self._dispatcher,
                job=self,
                proc_time=op_proc_time,
                machine_identifier=machine_order[idx],
            )
            self.operations.append(Op)
            
        self.open_operations = self.operations.copy()
        self.total_num_ops: int = len(self.operations)
        self.num_finished_ops: int = 0
        self._current_op: Operation = self.operations[0]
        
        ### STATE ###
        # intra-process job state parameters
        # job is being processed, maybe better naming in future
        self.is_occupied: bool = False
        # waiting state only when released
        self.is_waiting: bool = False
        # if lying on failed machine
        self.is_failed: bool = False
        
        # intra-process time characteristics
        self.time_occupied: float = 0.
        self.time_waiting: float = 0.
        self.time_failed: float = 0.
        
        # inter-process job state parameters
        # first operation scheduled --> released job
        self.is_released: bool = False
        # job's next operation is disposable
        # true for each new job, maybe reworked in future for jobs with
        # a start date later than creation date
        self.is_disposable: bool = True
        # add job to disposable ones
        ret = self.dispatcher.add_disposable_job(self)
        # last operation ended --> finished job
        self.is_finished: bool = False
        
        # inter-process time characteristics
        # time of first operation starting point
        self.time_entry: float = 0.
        # time of last operation ending point
        self.time_exit: float = 0.
        
        # current resource location
        self.current_resource: object | None = None # specify type if class definition finished
    
    @property
    def dispatcher(self) -> Dispatcher:
        return self._dispatcher
    
    @property
    def job_id(self) -> JobID:
        return self._job_id
    
    @property
    def current_op(self) -> Operation:
        """
        returns the current operation of the job
        If a job is currently being processed its current operation is 
        not changed until this operation is finished.
        """
        return self._current_op
    
    def get_next_operation(self) -> Operation:
        """
        get next operation
        """
        op = self.open_operations.popleft()
        self._current_op = op
        
        return op
    
    def process(self) -> None:
        # perform routing as long as there are open operations
        # perhaps problem with sinks because there are no final operations for sinks
        while len(self.open_operations) != 0:
                # get current operation
                curr_op = self.get_next_operation()
                
                ### REQUEST TO DISPATCHER (JOB SIDE: Allocation)
                # --> machine instance
                # target machine is more like a machine group with parallel machines
                machine = curr_op.target_machine
                
                print(f"Try putting Job-ID {self.job_id} \t Operation with ID {curr_op.op_id} \
                    on machine {machine}")
                
                #machine = self.dispatcher.get_machine_instance_by_id(machine_id)
                #print(f"Machine instance is {machine}")
                #print(f"Machine instance is: {machine} and ispassive {machine.ispassive()}")
                
                
                # entering associated resource queue
                self.enter(machine.buffer)
                if machine.ispassive():
                    machine.activate()
                yield self.passivate()
                # wait for processing
                #print(f"[END] Time now Job-ID {self.job_id} is {self.env.now()}")
                
                # after processing: write operation data
                # resource name, start, end, job name
                self._dispatcher.write_op_times(op_id=curr_op.op_id)

**Link Collection**

- [Environment](#environment)
- [Machine](#machine)
- [Dispatcher](#dispatcher)
- [Operation](#operation)
- [Job](#job)
- [Logic Test](#logic_test)


[Jump to top](#top)

---
#### Test simulation logic
- first main goal: execution of a generic example in a simple job shop with single machines and FIFO order

In [114]:
class TestDispatcher():
    def __init__(self) -> None:
        pass
    
    def add_dicts(self, job_dict, machine_dict):
        self.job_dict = job_dict
        self.machine_dict = machine_dict
        
    def get_machine_instance_by_id(self, ident):
        return self.machine_dict[ident]

In [115]:
f"buffer_{MachInst.name()}"

'buffer_M_env_2'

In [39]:
class TestMachine(sim.Component):
    def __init__(self, machine_id, environment, *args, **kwargs):
        super().__init__(env=environment, *args, **kwargs)
        self.machine_id = machine_id
        self.buffer = sim.Queue(env=environment)
        
    def process(self):
        while True:
            #yield self.passivate
            if len(self.buffer) == 0:
                yield self.passivate
            print("Machine is getting job from queue")
            # theoretically request to dispatcher possible
            job = self.buffer.pop()
            pro_time = job.pro_time
            print(f"Machine {self} got Job with ID {job.job_id} and ProcTime {pro_time}")
            #print(f"Time is {self.env.now()}")
            # processing
            self.hold(job.pro_time)
            job.activate()
    
        
class TestJob(sim.Component):
    def __init__(self, job_id, ops_proc_times, ops_machine_list, dispatcher, environment, machine_list, *args, **kwargs):
        super().__init__(env=environment, *args, **kwargs)
        self.job_id = job_id
        self.ops_proc_times = ops_proc_times.copy()
        self.ops_machine_list = ops_machine_list.copy()
        self.dispatcher = dispatcher
        self.ops_counter = 0
        self.pro_time = 5
        self.machine_list = machine_list.copy()
        
    def add_dicts(self, machine_dict):
        #self.job_dict = job_dict
        self.machine_dict = machine_dict
        
    def process(self):
        print(f"Job-ID {self.job_id} starts {env.now()}")
        while self.ops_counter < (len(self.ops_proc_times) - 1):
            print("Get Next operation...")
            #self.pro_time = self.ops_proc_times[self.ops_counter]
            machine_id = self.ops_machine_list[self.ops_counter]
            print(f"Operation with ProTime {self.pro_time} on machine {machine_id}")
            print("Obtain machine instance...")
            machine = self.dispatcher.get_machine_instance_by_id(machine_id)
            #machine = self.machine_dict[machine_id]
            #machine = self.machine_list.pop(0)
            print(f"Machine instance is: {machine} and ispassive {machine.ispassive()}")
            
            # now enter buffer
            self.enter(machine.buffer)
            # activate machine if passive
            #if machine.ispassive():
            machine.activate()
            yield self.passivate()
            
            
            self.ops_counter += 1
            print(f"Job-ID {self.job_id} ends {self.env.now()}")

In [40]:
test_dispatcher = TestDispatcher()

In [41]:
test_ProcTimes = mat_ProcTimes.copy()
#test_ProcTimes = np.expand_dims(test_ProcTimes, 0)

test_JobMachID = mat_JobMachID.copy()
#test_JobMachID = np.expand_dims(test_JobMachID, 0)

In [42]:
class TestMachine(sim.Component):
    def __init__(self, machine_id, env, *args, **kwargs):
        super().__init__(env=env, *args, **kwargs)
        self.machine_id = machine_id
        self.buffer = sim.Queue(env=env)
    
    def process(self):
        while True:
            if len(self.buffer) == 0:
                yield self.passivate()
            job = self.buffer.pop()
            proc_time = job.proc_time
            print(f"[START] job ID {job.job_id} at {self.env.now()} on machine ID {self.machine_id} with proc time {job.proc_time}")
            yield self.hold(job.proc_time)
            print(f"[END] job ID {job.job_id} at {self.env.now()} on machine ID {self.machine_id}")
            job.activate()

class TestJob(sim.Component):
    def __init__(self, ident, machine_order, operation_procs, env, dispatcher, *args, **kwargs):
        super().__init__(env=env, *args, **kwargs)
        #self.machine_id = machine_id
        self.machine_list = machine_list.copy()
        self.proc_time = None
        self.counter = 0
        self.operation_procs = operation_procs.copy()
        #self.machine_dict = machine_dict.copy()
        #self.dict_keys = list(machine_dict.keys())
        self.machine_order = machine_order.copy()
        self.job_id = ident
        self.dispatcher = dispatcher
        
    def process(self):
        while len(self.operation_procs) != 0:
            #print(f"Job-ID {self.job_id} starts {env.now()}")
            print(f"Job-ID {self.job_id} \t Operation with ID {self.counter}")
            self.proc_time = self.operation_procs.pop(0)
            #machine = machine_list.pop(0)
            #machine = self.machine_dict[self.dict_keys[self.counter]]
            machine_id = self.machine_order.pop(0)
            machine = self.dispatcher.get_machine_instance_by_id(machine_id)
            #print(f"Machine instance is {machine}")
            #print(f"Machine instance is: {machine} and ispassive {machine.ispassive()}")
            self.enter(machine.buffer)
            if machine.ispassive():
                machine.activate()
            yield self.passivate()
            #print(f"[END] Time now Job-ID {self.job_id} is {self.env.now()}")
            self.counter += 1

In [43]:
env = sim.Environment(trace=False)
machine_db = dict()
job_db = dict()
machine_list_inst = list()

for machine_id in np.unique(mat_JobMachID):
    machine = TestMachine(machine_id=machine_id.item(), env=env)
    machine_db[machine_id.item()] = machine
    machine_list_inst.append(machine)

for job_id, (proc_list, machine_list) in enumerate(zip(test_ProcTimes, test_JobMachID)):
    job = TestJob(ident=job_id, machine_order=machine_list.tolist(), operation_procs=proc_list.tolist(), 
                  dispatcher=test_dispatcher, env=env)
    #job.add_dicts(machine_db)
    job_db[job_id] = job
"""
proc_list = [10,8,6]
machine_list = [0,1,2]
job_id = 0
job = TestJob(ident=job_id, machine_dict=machine_db, operation_procs=proc_list,
                  dispatcher=test_dispatcher, env=env)
"""
   
test_dispatcher.add_dicts(job_db, machine_db)


In [44]:
ret = job_db[0]

<a id='logic_test'></a>
#### Test Logic with integrated classes

In [206]:
(n_jobs, n_machines, n_ops, 
 mat_ProcTimes, mat_JobMachID, mat_OpID) = gen_rnd_JSSP_inst(5,3)

In [207]:
env = SimulationEnvironment(name='base')

for machine in np.unique(mat_JobMachID):
    MachInst = Machine(env=env, custom_identifier=machine.item())

dispatcher = Dispatcher(env=env)

INFO:base:Successfully registered object with EnvID 0 and name M_env_0
INFO:base:Successfully registered object with EnvID 1 and name M_env_1
INFO:base:Successfully registered object with EnvID 2 and name M_env_2
INFO:base:Successfully registered dispatcher with EnvID 3


In [208]:
test_ProcTimes = mat_ProcTimes.copy()
#test_ProcTimes = mat_ProcTimes[0,:]
#test_ProcTimes = np.expand_dims(test_ProcTimes, 0).copy()

test_JobMachID = mat_JobMachID.copy()
#test_JobMachID = mat_JobMachID[0,:]
#test_JobMachID = np.expand_dims(test_JobMachID, 0).copy()

In [209]:
ret = dispatcher.gen_job_pool_generic(
    mat_ProcTimes=test_ProcTimes,
    mat_JobMachID=test_JobMachID,
)

INFO:base:Successfully registered job with JobID 0 and name J_gen_0
INFO:base:Successfully registered operation with OpID 0 and name O_gen_0
INFO:base:Successfully registered operation with OpID 1 and name O_gen_1
INFO:base:Successfully registered operation with OpID 2 and name O_gen_2
INFO:base:Successfully registered job with JobID 1 and name J_gen_1
INFO:base:Successfully registered operation with OpID 3 and name O_gen_3
INFO:base:Successfully registered operation with OpID 4 and name O_gen_4
INFO:base:Successfully registered operation with OpID 5 and name O_gen_5
INFO:base:Successfully registered job with JobID 2 and name J_gen_2
INFO:base:Successfully registered operation with OpID 6 and name O_gen_6
INFO:base:Successfully registered operation with OpID 7 and name O_gen_7
INFO:base:Successfully registered operation with OpID 8 and name O_gen_8
INFO:base:Successfully registered job with JobID 3 and name J_gen_3
INFO:base:Successfully registered operation with OpID 9 and name O_gen_

In [210]:
dispatcher.op_db

,job_id,job_name,custom_id,op,name,machine,machine_name,duration,start_time,end_time,status
op_id,,,,,,,,,,,
0,0,J_gen_0,None,<__main__.Operation object at 0x00000268CA8D5990>,O_gen_0,Machine (M_env_2),M_env_2,2.0,0.0,0.0,None
1,0,J_gen_0,None,<__main__.Operation object at 0x00000268CAA253D0>,O_gen_1,Machine (M_env_0),M_env_0,1.0,0.0,0.0,None
2,0,J_gen_0,None,<__main__.Operation object at 0x00000268CA890C10>,O_gen_2,Machine (M_env_1),M_env_1,9.0,0.0,0.0,None
3,1,J_gen_1,None,<__main__.Operation object at 0x00000268CA77C5D0>,O_gen_3,Machine (M_env_0),M_env_0,7.0,0.0,0.0,None
4,1,J_gen_1,None,<__main__.Operation object at 0x00000268CA625B90>,O_gen_4,Machine (M_env_2),M_env_2,9.0,0.0,0.0,None
5,1,J_gen_1,None,<__main__.Operation object at 0x00000268CA97D150>,O_gen_5,Machine (M_env_1),M_env_1,6.0,0.0,0.0,None
6,2,J_gen_2,None,<__main__.Operation object at 0x00000268CAA2E7D0>,O_gen_6,Machine (M_env_2),M_env_2,4.0,0.0,0.0,None
7,2,J_gen_2,None,<__main__.Operation object at 0x00000268CA941CD0>,O_gen_7,Machine (M_env_1),M_env_1,4.0,0.0,0.0,None
8,2,J_gen_2,None,<__main__.Operation object at 0x00000268CAA03350>,O_gen_8,Machine (M_env_0),M_env_0,1.0,0.0,0.0,None


In [211]:
env.run()

Try putting Job-ID 0 	 Operation with ID 0                     on machine Machine (M_env_2)
Try putting Job-ID 1 	 Operation with ID 3                     on machine Machine (M_env_0)
Try putting Job-ID 2 	 Operation with ID 6                     on machine Machine (M_env_2)
Try putting Job-ID 3 	 Operation with ID 9                     on machine Machine (M_env_1)
Try putting Job-ID 4 	 Operation with ID 12                     on machine Machine (M_env_0)
Machine 2 is getting job from queue
[START] job ID 0 at 0.0 on machine ID 2                 with proc time 2
Machine 0 is getting job from queue
[START] job ID 1 at 0.0 on machine ID 0                 with proc time 7
Machine 1 is getting job from queue
[START] job ID 3 at 0.0 on machine ID 1                 with proc time 4
[END] job ID 0 at 2.0 on machine ID 2
Machine 2 is getting job from queue
[START] job ID 2 at 2.0 on machine ID 2                 with proc time 4
Try putting Job-ID 0 	 Operation with ID 1                     on

In [212]:
dispatcher.op_db

,job_id,job_name,custom_id,op,name,machine,machine_name,duration,start_time,end_time,status
op_id,,,,,,,,,,,
0,0,J_gen_0,None,<__main__.Operation object at 0x00000268CA8D5990>,O_gen_0,Machine (M_env_2),M_env_2,2.0,0.0,2.0,None
1,0,J_gen_0,None,<__main__.Operation object at 0x00000268CAA253D0>,O_gen_1,Machine (M_env_0),M_env_0,1.0,12.0,13.0,None
2,0,J_gen_0,None,<__main__.Operation object at 0x00000268CA890C10>,O_gen_2,Machine (M_env_1),M_env_1,9.0,13.0,22.0,None
3,1,J_gen_1,None,<__main__.Operation object at 0x00000268CA77C5D0>,O_gen_3,Machine (M_env_0),M_env_0,7.0,0.0,7.0,None
4,1,J_gen_1,None,<__main__.Operation object at 0x00000268CA625B90>,O_gen_4,Machine (M_env_2),M_env_2,9.0,7.0,16.0,None
5,1,J_gen_1,None,<__main__.Operation object at 0x00000268CA97D150>,O_gen_5,Machine (M_env_1),M_env_1,6.0,22.0,28.0,None
6,2,J_gen_2,None,<__main__.Operation object at 0x00000268CAA2E7D0>,O_gen_6,Machine (M_env_2),M_env_2,4.0,2.0,6.0,None
7,2,J_gen_2,None,<__main__.Operation object at 0x00000268CA941CD0>,O_gen_7,Machine (M_env_1),M_env_1,4.0,6.0,10.0,None
8,2,J_gen_2,None,<__main__.Operation object at 0x00000268CAA03350>,O_gen_8,Machine (M_env_0),M_env_0,1.0,14.0,15.0,None


In [51]:
machine = env.get_res_obj_by_prop(property='custom_id', val=2)

In [52]:
job = dispatcher.get_job_obj_by_prop('job_id', val=0)

In [53]:
op_test = job.operations[2]

In [54]:
op_test.proc_time

9

In [213]:
dispatcher.job_db

,custom_id,job,name,job_type,status
job_id,,,,,
0,None,Job (J_gen_0),J_gen_0,Job,None
1,None,Job (J_gen_1),J_gen_1,Job,None
2,None,Job (J_gen_2),J_gen_2,Job,None
3,None,Job (J_gen_3),J_gen_3,Job,None
4,None,Job (J_gen_4),J_gen_4,Job,None


In [214]:
fig = dispatcher.draw_gantt_chart(sort_by_machine_name=True)

##### Visualisation

In [142]:
import plotly.express as px
import datetime

In [164]:
dft_year = 1970
dft_month = 1
dft_day = 1

In [165]:
start_time = 0
end_time = 5
date_obj_start = datetime.datetime(dft_year, dft_month, dft_day, hour=start_time)
date_obj_end = datetime.datetime(dft_year, dft_month, dft_day, hour=end_time)

In [173]:
df = dispatcher.op_db

In [174]:
df

,job_id,job_name,custom_id,op,name,machine,machine_name,duration,start_time,end_time,status
op_id,,,,,,,,,,,
0,0,J_gen_0,None,<__main__.Operation object at 0x00000268CA5F8FD0>,O_gen_0,Machine (M_env_2),M_env_2,2.0,0.0,2.0,None
1,0,J_gen_0,None,<__main__.Operation object at 0x00000268C8594D90>,O_gen_1,Machine (M_env_0),M_env_0,1.0,7.0,8.0,None
2,0,J_gen_0,None,<__main__.Operation object at 0x00000268C85C3050>,O_gen_2,Machine (M_env_1),M_env_1,9.0,16.0,25.0,None
3,1,J_gen_1,None,<__main__.Operation object at 0x00000268CA605650>,O_gen_3,Machine (M_env_0),M_env_0,7.0,0.0,7.0,None
4,1,J_gen_1,None,<__main__.Operation object at 0x00000268CA60FFD0>,O_gen_4,Machine (M_env_1),M_env_1,9.0,7.0,16.0,None
5,1,J_gen_1,None,<__main__.Operation object at 0x00000268CA603D50>,O_gen_5,Machine (M_env_2),M_env_2,6.0,16.0,22.0,None


In [175]:
test = df.filter(items=['job_name', 'machine_name', 'start_time', 'end_time', 'duration'])

In [176]:
test

,job_name,machine_name,start_time,end_time,duration
op_id,,,,,
0,J_gen_0,M_env_2,0.0,2.0,2.0
1,J_gen_0,M_env_0,7.0,8.0,1.0
2,J_gen_0,M_env_1,16.0,25.0,9.0
3,J_gen_1,M_env_0,0.0,7.0,7.0
4,J_gen_1,M_env_1,7.0,16.0,9.0
5,J_gen_1,M_env_2,16.0,22.0,6.0


In [177]:
test['delta'] = test['end_time'] - test['start_time']

In [178]:
test = test.sort_values('machine_name')

In [151]:
type(fig)

plotly.graph_objs._figure.Figure

In [179]:
fig = px.timeline(test, x_start='start_time', x_end='end_time', y='machine_name', color='job_name')
fig.update_yaxes(autorange='reversed')
fig.layout.xaxis.type = 'linear'

for d in fig.data:
    filt = test['job_name'] == d.name
    #d.x = test[filt]['delta']
    d.x = test.loc[filt, 'duration']

fig.show()

In [180]:
fig.data

(Bar({
     'alignmentgroup': 'True',
     'base': array([7.0, 16.0, 0.0], dtype=object),
     'hovertemplate': ('job_name=J_gen_0<br>start_time' ... 'chine_name=%{y}<extra></extra>'),
     'legendgroup': 'J_gen_0',
     'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
     'name': 'J_gen_0',
     'offsetgroup': 'J_gen_0',
     'orientation': 'h',
     'showlegend': True,
     'textposition': 'auto',
     'x': array([1., 9., 2.]),
     'xaxis': 'x',
     'y': array(['M_env_0', 'M_env_1', 'M_env_2'], dtype=object),
     'yaxis': 'y'
 }),
 Bar({
     'alignmentgroup': 'True',
     'base': array([0.0, 7.0, 16.0], dtype=object),
     'hovertemplate': ('job_name=J_gen_1<br>start_time' ... 'chine_name=%{y}<extra></extra>'),
     'legendgroup': 'J_gen_1',
     'marker': {'color': '#EF553B', 'pattern': {'shape': ''}},
     'name': 'J_gen_1',
     'offsetgroup': 'J_gen_1',
     'orientation': 'h',
     'showlegend': True,
     'textposition': 'auto',
     'x': array([7., 9., 6.]),
    

In [217]:
# https://stackoverflow.com/questions/66078893/plotly-express-timeline-for-gantt-chart-with-integer-xaxis
df = pd.DataFrame([
    dict(Resource="M1", Start=0, Finish=10, Job="J1"),
    dict(Resource="M2", Start=0, Finish=5, Job="J2"),
    dict(Resource="M1", Start=11, Finish=15, Job="J2")
])
df['delta'] = df['Finish'] - df['Start']

fig = px.timeline(df, x_start='Start', x_end='Finish', y='Resource', color='Job')
fig.update_yaxes(autorange='reversed')
fig.layout.xaxis.type = 'linear'

for d in fig.data:
    filt = df['Job'] == d.name
    d.x = df[filt]['delta']

fig.show()


In [208]:
df

,Resource,Start,Finish,Job,delta
0,M1,0,10,J1,10
1,M2,0,5,J2,5
2,M1,11,15,J2,4


In [214]:
for d in fig.data:
    print(d.name)
    filt = df['Job'] == d.name
    d.x = df[filt]['delta']
    print(d.x)

J1
[10]
J2
[5 4]


In [213]:
fig.data[0].x = df[filt]['delta']

In [201]:
fig.data[0]

Bar({
    'alignmentgroup': 'True',
    'base': array([0], dtype=object),
    'hovertemplate': 'Job=J1<br>Start=%{base}<br>Finish=%{x}<br>Resource=%{y}<extra></extra>',
    'legendgroup': 'J1',
    'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
    'name': 'J1',
    'offsetgroup': 'J1',
    'orientation': 'h',
    'showlegend': True,
    'textposition': 'auto',
    'x': array([1.e-05]),
    'xaxis': 'x',
    'y': array(['M1'], dtype=object),
    'yaxis': 'y'
})

In [184]:
import plotly.express as px
import pandas as pd

df = pd.DataFrame([
    dict(Resource="Job A", Start=1, Finish=4),
    dict(Resource="Job B", Start=2, Finish=6),
    dict(Resource="Job A", Start=4, Finish=10)
])
df['delta'] = df['Finish'] - df['Start']

fig = px.timeline(df, x_start="Start", x_end="Finish", y="Resource")
fig.update_yaxes(autorange="reversed") 

fig.layout.xaxis.type = 'linear'
fig.data[0].x = df.delta.tolist()
#f = fig.full_figure_for_development(warn=False)
fig.show()

In [177]:
df

,Resource,Start,Finish,Job,delta
0,M1,0,10,J1,10
1,M2,0,5,J2,5
2,M1,10,15,J2,5


In [170]:
import plotly.figure_factory as ff

df = pd.DataFrame([
    dict(Task="Job A", Start=0, Finish=10, Resource="Alex"),
    dict(Task="Job B", Start=12, Finish=24, Resource="Alex"),
    dict(Task="Job C", Start=5, Finish=20, Resource="Max")
])

fig = ff.create_gantt(df, index_col = 'Resource',  bar_width = 0.4, show_colorbar=True)
fig.update_layout(xaxis_type='linear', autosize=True, width=800, height=400)
fig.show()

In [159]:
df

,Task,Start,Finish,Resource
0,M1,2009-01-01,2009-02-28,J1
1,M1,2009-03-03,2009-03-28,J2
2,M2,2009-03-05,2009-04-15,J2
3,M3,2009-02-20,2009-05-30,J3


**Link Collection**

- [Environment](#environment)
- [Machine](#machine)
- [Dispatcher](#dispatcher)
- [Operation](#operation)
- [Job](#job)
- [Logic Test](#logic_test)


[Jump to top](#top)